# Nødvendige biblioteker

In [13]:
import requests
import ftplib
import pandas as pd
import numpy as np
import tqdm
import time
from io import BytesIO
from bs4 import BeautifulSoup as bs

# Opsætning af ftp serveradresse og bibliotek

In [14]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"

# Initialisering af lister
## Jeg skal finde ud af, hvordan jeg dropper dem igen, når jeg er færdig

In [15]:
dirs = []
sub_dir = []
filer = []
filer_navne = []
filer_size = []
sti_navn = []
indhold = []
suppe = []

# Login til ftp serveren og henter en liste over biblioteker

In [16]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

# Bygger en liste med underbibliotekter

In [17]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

# Bygger en liste med filnavne

In [18]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(filer.append)

# Ekstrakt af filnavn

In [19]:
for i in range(0, len(filer)):
    filer_navne.append(filer[i][39:])

# Opbygning af hele stien og filnavn

In [20]:
for i in range(0, len(filer_navne)):
    sti_navn.append(ftp_dir+filer_navne[i][0:5]+'/'+filer_navne[i])

# Overførsel af de enkelte filer fra ftp://oda.ft.dk, og indholdet skrives til liste

In [21]:
for i in tqdm.tqdm(range(0, len(sti_navn))):
# for i in tqdm.tqdm(range(0, 10)):
    r = BytesIO()
    ftp.retrbinary(f"RETR {sti_navn[i]}", r.write)
    indhold.append(r.getvalue())
    r.close()

100%|██████████| 1489/1489 [03:01<00:00,  8.19it/s]


In [30]:
indhold[0]

b'\xef\xbb\xbf<?xml version="1.0" encoding="utf-8"?><Dokument edixistatus=""><MetaMeeting><ParliamentarySession>20091</ParliamentarySession><ParliamentaryGroup>Folketinget</ParliamentaryGroup><MeetingNumber>100</MeetingNumber><DateOfSitting>2010-05-31T10:00:00</DateOfSitting><Location>Folketingssalen</Location><EdixiDocLocation>W:\\20091\\M100\\</EdixiDocLocation><AudioFileFolder>X:\\DAudio\\FT2009\\100\\SALEN\\</AudioFileFolder></MetaMeeting><TitelGruppe><Titel><Linea><Char>100. m\xc3\xb8de</Char></Linea></Titel><UnderTitel><Linea><Char>Mandag den 31. maj 2010 kl. 10.00</Char></Linea></UnderTitel></TitelGruppe><DagsordenPlan><Rubrica>\r\n  <Linea>\r\n    <Char>Dagsorden</Char>\r\n  </Linea>\r\n</Rubrica>\r\n<PunktTekst>\r\n  <Exitus>\r\n    <Linea>\r\n      <Char>1) 1. behandling af lovforslag nr. L 222:</Char>\r\n    </Linea>\r\n  </Exitus>\r\n  <Exitus>\r\n    <Linea>\r\n      <Char>Forslag til lov om \xc3\xa6ndring af lov om arbejdsl\xc3\xb8shedsforsikring m.v. (Neds\xc3\xa6ttelse 

# BeautifulSoup - her kommer der korrekte danske karakterer, og der sker eet eller andet med XML-tags'ene
## Men det er tilsyneladende ikke alle æøå, den kan finde ud af

In [22]:
for i in tqdm.tqdm(range (0,10)):
    suppe.append(bs(indhold[i],"lxml"))

100%|██████████| 10/10 [00:05<00:00,  1.73it/s]


# Dataframe skabes

In [23]:
df_suppe = pd.DataFrame(suppe, columns = ['metadata', 'indhold'])

/Users/dana_tiger/opt/anaconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


# Der søges på de møder i salen, hvor ordet "sygeplejerske" har været nævnt
# Der returneres 512 møder i salen, hvilket svarer til ca. 35%

In [24]:
substring = 'sygeplejerske'
df_sygep = df_suppe[df_suppe.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]

In [25]:
df_suppe.to_csv('suppe.csv') 

In [26]:
df_sygep.to_csv('sygep.csv') 

In [27]:
pd.read_csv('suppe.csv')

,Unnamed: 0,metadata,indhold
0,0,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
1,1,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
2,2,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
3,3,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
4,4,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
5,5,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
6,6,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
7,7,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
8,8,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
9,9,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."


In [28]:
pd.read_csv('sygep.csv')

,Unnamed: 0,metadata,indhold
0,0,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
1,2,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
2,3,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
3,6,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
4,7,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
5,9,"xml version=""1.0"" encoding=""utf-8""?","<html><body><dokument edixistatus=""""><metameet..."
